In [8]:
%matplotlib tk
import numpy as np
import hyperspy.api as hs
import pycrystem as pc
import pymatgen as pmg
from pymatgen.transformations.standard_transformations import DeformStructureTransformation
from pycrystem.indexation_generator import IndexationGenerator
from scipy.constants import pi

This work is based off of a notebook created by Duncan Johnstone for the Trondheim Diffraction Workshop in Summer 2017

In [2]:
## Creating our structure
si = pmg.Element("Si")
lattice = pmg.Lattice.cubic(5.431)
structure = pmg.Structure.from_spacegroup("Fd-3m",lattice, [si], [[0, 0, 0]])

## Creating our electron diffraction set up
ediff = pc.ElectronDiffractionCalculator(300., 0.025)

Define a series of affine transformations from 0-2%

In [5]:
affines = []
for s in np.linspace(0,0.02,11):
    affine_row = [[1+s,0,0],[0,1,0],[0,0,1]]
    affines.append(affine_row)            

In [13]:
data = []
for affine in affines:
    deform = DeformStructureTransformation(affine)
    strained = deform.apply_transformation(structure)
    diff_dat = ediff.calculate_ed_data(strained, 2.5)
    dpi = diff_dat.as_signal(512, 0.02, 2.5)
    data.append(dpi.data)
strain_series = pc.ElectronDiffraction(data)

#stack multiple versions of tilt series together so that you have repeats of measurements for error analysis
"""This seems to be some sort of important witchcraft..."""
dp = hs.stack((strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series))


In [14]:
dp.plot()